<center><img src="https://img-blog.csdnimg.cn/43aebc53f7044cbbb657dc48c456be61.jpeg" width="100%"></center>

# 深度学习计算

## 一、层与块

为了实现这些复杂的网络，我们引入了<font color="red">**神经网络块**</font>的概念。使用块进行抽象的一个好处就是可以递归的将一些块组合成更大的组件。因此，我们可以通过定义代码来按需生成任意复杂度的块，已达到通过简洁的代码实现复杂的神经网络。

从变成角度来看，块由类（class）表示。它的任何子类都必须定义一个将其输入转换为输出的前向传播函数，并且必须存储任何必要的参数。另外，为了计算梯度，块必须具有反向传播函数。在定义我们自己的块时，由于自动微分提供了一些后端实现，因此在实际编程过程中我们只需要考虑前向传播函数和必要的参数即可。

下面的代码生成一个网络，其中包括20个输入特征，和一个具有256个单元和ReLU激活函数的全连接隐藏层，然后是一个具有10个隐藏单元且不带激活函数的全连接输出层。

In [20]:
import torch
from torch import nn
from torch.nn import functional as F

# 定义网络
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

# 生成两个随机输入数据
X = torch.rand(2, 20)

# 由网络生成输出数据
net(X)

tensor([[-8.9494e-02, -6.8739e-02,  2.0899e-01,  2.3141e-01, -3.8358e-01,
         -1.7187e-02, -1.1569e-01,  1.9666e-01, -2.0721e-01, -2.0310e-01],
        [ 4.4047e-02, -4.7203e-02,  2.7837e-01,  3.0087e-01, -4.3936e-01,
          8.8485e-02, -1.0837e-01,  2.0582e-01,  4.3734e-04, -2.0276e-01]],
       grad_fn=<AddmmBackward0>)

在上面的代码中，我们通过实例化`nn.Sequential`来构建我们的模型，层的执行顺序是作为参数传递的。简而言之，`nn.Sequential`定义了一个特殊的`Module`，它就是Pytorch中的一个块类，并且它维护了一个由`Module`组成的有序列表。

**注意**：两个全连接层都是`Linear`类的实例，`Linear`类本身就是`Module`的子类。

### 1.1 自定义块

在实现自定义块之前，我们需要明白每个块必须提供的基本功能：

- 1、将输入数据作为其前向传播函数的参数。
- 2、通过前向传播函数来生成输出。
- 3、计算其输出关于输入的梯度，可通过其反向传播函进行访问，且这个过程通常是自动发生的。
- 4、存储和访问前向传播计算所需要的参数。
- 5、根据需要初始化模型参数。

在下面的代码片段中，我们从零开始编写一个块。它包含一个多层感知机，其具有256个隐藏层和一个10维输出层。

注意，下面的MLP类继承了表示块的类，我们实现只需要提供自己的（1）构造函数（Python中的`__init__`函数）和（2）前向传播函数。

**自定义块**的代码如下所示：

In [21]:
class MLP(nn.Module):
    # 使用模型参数声明两个全连接层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化
        super().__init__()
        # 隐藏层的实例化
        self.hidden = nn.Linear(20, 256)
        # 输出层的实例化
        self.out = nn.Linear(256, 10)

    # 定义模型的前向传播：根据输入X返回所需要的模型输出
    def forward(self, X):
        # 注意，这里的激活函数使用的是nn.functional模块中定义的ReLU函数
        return self.out(F.relu(self.hidden(X)))

可以通过`MLP()`查看定义的MLP块的结构：

In [22]:
MLP()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=10, bias=True)
)

**关于`MLP()`块的说明：**

- 对于其前向传播函数，其输入$X$作为其输入，然后计算带有激活函数的隐藏表示并输出未规范化的输出值。在这个MLP实现过程中，两个层都是实例变量。
- 接着我们实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层。 注意一些关键细节： 首先，我们定制的`__init__`函数通过`super().__init__()` 调用父类的`__init__`函数， 省去了重复编写模版代码的痛苦。 然后，我们实例化两个全连接层， 分别为`self.hidden`和`self.out`。 注意，除非我们实现一个新的运算符， 否则我们不必担心反向传播函数或参数初始化， 系统将自动生成这些。

调用`MLP()`的方法如下所示：

In [23]:
# 实例化一个MLP()网络net
net = MLP()

# 输入X，并得到输出结果
net(X)

tensor([[ 0.3217,  0.1555, -0.0955,  0.0448, -0.1109, -0.0314, -0.0972, -0.1372,
          0.0712,  0.3205],
        [ 0.2880,  0.0086, -0.0439,  0.2506, -0.2628, -0.2156,  0.0119, -0.0643,
          0.0160,  0.1856]], grad_fn=<AddmmBackward0>)

> 块的一个主要**优点**就是它的<font color="red">多功能性</font>，我们可以通过块创建层（比如全连接层）、整个模型（比如上面创建MLP类）或者具有中等复杂度的各种组建。

### 1.2 顺序块

首先我们先仔细了解一下`Sequential`类的工作原理，总的来说它的**作用**是：<font color="red">将其他模块串联起来。</font>为了构建自己的顺序块`MySequential`，我们需要定义两个**关键函数**：

- 1、一个将块逐个添加到列表中的函数；
- 2、一个前向传播函数，用于将输入按添加块的顺序传递给块组成的网络链条。

下面定义的顺序块`MySequential`类与默认的`Sequential`类具有相同的功能，通过下面的代码我们可以很好的大体理解循序块`Sequential`的运行机制。

In [24]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # module是Module子类的一个实例，此处将其保存在Module类成员变量_modules（其类型为OrderedDict）中。
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

> - 由上面的代码可以看出，`__init__`函数将每个模块逐个添加到有序字典`_modules`中。也就是说每个`Module`都有一个`_modules`属性，这种机制的好处就在于：在模块的参数初始化过程中，系统知道在`_modules`字典中查找需要初始化参数的子块。

当`MySequential`的前向传播函数被调用时，每个添加的块都按照他们被添加的顺序执行。下面通过构建`MySequential`类重新实现多层感知机，代码如下所示：

In [25]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net

MySequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)

### 1.3 在前向传播函数中执行代码

##### 场景分析：

`Sequential`类使得模型构造变得非常简单，并允许我们在不必定义自己的类的情况下组合新的架构。**然而**，并不是所有的架构都是简单顺序架构。当需面临更加灵活的实际场景时，我们还是需要定义自己的块才能满足需求。比如以下两种场景：
- **（1）有的时候我们可能希望在前向传播函数中执行Python的控制流；**
- **（2）执行任意的数学运算，而不是简单地以来预定义的神经网络层**

##### 场景实例：
（1）通常情况下，网络中的所有操作都对网络的激活值及网络的参数起作用。然而，有的时候我们可能希望合并既不是**上一层的结果**也不是**可更新的参数项**，这也就是所谓的<font color="red">**常数参数（constant parameter）**</font>。比如，我们需要如下计算函数的层：
$$f(\mathbf{x, w})=c \cdot \mathbf{w^\top x},$$
其中，$\mathbf{x}$表示输入，$\mathbf{w}$为参数，$c$表示在某个优化过程中没有更新的指定常量。对于这种情况，我们可以通过实现如下所示的`FixedHiddenMLP`类来满足实际需求：

In [26]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数，因此在其训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层，即两个全连接层共享参数
        X = self.linear(X)
        # 控制流：如果输出的L1范数大于1，则将输出向量除以2,直至其范数小1终止循环（只是用于展示用，实际情况这种处理方法不太常用）
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

上面的`FixedHiddenMLP`模型实现了一个隐藏层，其权重在实例化时被随机初始化，由于设置了`requires_grad=False`，所以之后就成为了常量。`self.rand_weight`权重不是一个模型参数，因此它永远不会被反向传播更新。然后，神经网络将这个固定层的输出传入一个全连接层。

## 二、参数管理

深度学习参数管理主要包括以下三方面内容：

- 访问参数，用于测试、诊断与可视化；
- 参数初始化；
- 在不同模型组件间共享参数。

### 2.1 参数访问

从已经有模型中访问参数。当通过`Sequential`类定义模型时，我们可以通过索引来访问模型的任意层。构建的模型就像一个列表一样，没层的参数都在其属性中。如下代码所示，检查`net`网络的第二个全连接层（即第三个神经网络层）的参数：

In [38]:
# 首先构造网络net
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
# 构造输入X
X = torch.rand(size=(2, 4))

# 通过输出查看net的结构
print("神经网络的整体结构：\n", net)

# 输出第二个全连接层的参数
print("\n第二个全连接层的参数为：\n", net[2].state_dict())

神经网络的整体结构：
 Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

第二个全连接层的参数为：
 OrderedDict([('weight', tensor([[ 0.0095,  0.1977,  0.0934, -0.3136, -0.1743, -0.2168, -0.3238, -0.0382]])), ('bias', tensor([0.0115]))])


由上面代码的输出我们可以得到一些**重要的信息**：
- `net`网络的第二个全连接层包含两个参数，分别为<font color="red">权重</font>和<font color="red">偏置</font>。
- 权重和偏存储的数据格式为单精度浮点数（`float32`）。

#### 2.1.1 目标参数

> 注意：网络的每个参数都是参数类的一个实例。如果想要对参数执行任何操作，都需要访问底层的数值。

下面的代码从第二个全连接层提取偏置，提取后返回的是一个参数类实例，并进一步访问该参数的值。

In [46]:
print("第二个全连接层偏置参数的类型为：", type(net[2].bias))
print("\n第二个全连接层偏置参数的内容为：", net[2].bias)
print("\n第二个全连接层偏置参数的数值为：", net[2].bias.data)

第二个全连接层偏置参数的类型为： <class 'torch.nn.parameter.Parameter'>

第二个全连接层偏置参数的内容为： Parameter containing:
tensor([0.0115], requires_grad=True)

第二个全连接层偏置参数的数值为： tensor([0.0115])


> **总结：**
> - 参数是复合的对象，其包括值、梯度和额外信息，这就是为什么在实际应用需要对各种参数进行提取显式参数值操作的原因。
> - 另外，我们还可以访问每个参数的梯度，由于上面的网络并没有进行反向传播，因此参数的梯度还处于初始状态，我们可以通过下面的代码确认。

In [47]:
net[2].weight.grad == None

True

#### 2.1.2 一次性访问所有参数

当需要对所有参数执行操作时，逐个访问它们可能会很麻烦，而且当我们处理更为复杂的块（比如嵌套块）时，就更为复杂，因此我们需要递归整个树来提取每个子块的参数。下面的代码分别展示了访问第一个全连层参数和访问所有层参数的方法：

In [50]:
print("访问第一个全连接层参数：\n", *[(name, param) for name, param in net[0].named_parameters()])
print("\n访问所有层参数：\n", *[(name, param) for name, param in net.named_parameters()])

访问第一个全连接层参数：
 ('weight', Parameter containing:
tensor([[ 0.1789, -0.2886,  0.3663,  0.4276],
        [-0.3378, -0.3494,  0.1064,  0.3979],
        [ 0.1531,  0.1971,  0.0903, -0.3836],
        [-0.4974,  0.2669, -0.1556,  0.3825],
        [-0.3723,  0.3582,  0.4243,  0.1031],
        [-0.1792,  0.0565, -0.2381,  0.1089],
        [ 0.1850,  0.4989,  0.4927, -0.0389],
        [ 0.4200, -0.1241,  0.3632, -0.2877]], requires_grad=True)) ('bias', Parameter containing:
tensor([ 0.1429, -0.1209, -0.1820, -0.2926,  0.0566,  0.0505, -0.3493, -0.4406],
       requires_grad=True))

访问所有层参数：
 ('0.weight', Parameter containing:
tensor([[ 0.1789, -0.2886,  0.3663,  0.4276],
        [-0.3378, -0.3494,  0.1064,  0.3979],
        [ 0.1531,  0.1971,  0.0903, -0.3836],
        [-0.4974,  0.2669, -0.1556,  0.3825],
        [-0.3723,  0.3582,  0.4243,  0.1031],
        [-0.1792,  0.0565, -0.2381,  0.1089],
        [ 0.1850,  0.4989,  0.4927, -0.0389],
        [ 0.4200, -0.1241,  0.3632, -0.2877]], requires

另外，网络参数存储格式后，我们还可以通过下面的方法有针对性的访问网络参数：

In [53]:
net.state_dict()['2.bias']

tensor([0.0115])

#### 2.1.3 从嵌套块收集参数

对于网络具有多个块相互嵌套的时，结合下面的代码来理解参数命名约定的工作机制。下面的代码主要内容是首先定义了一个生成块函数，然后将这些块组合到更大的块中：

In [55]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    # 首先定义一个空的顺序块
    net = nn.Sequential()

    # 使用add_module类方法进行块的嵌套
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.0745],
        [0.0745]], grad_fn=<AddmmBackward0>)

查看`rgnet`的网络结构

In [56]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以可以通过像访问嵌套列表那样访问网络的目标参数。比如下面的代码是访问第一个主要块中、第二个子块的第一层偏置：

In [57]:
print("rgnet网络的第一个主要块中、第二个子块的第一层偏置为：\n", rgnet[0][1][0].bias.data)

rgnet网络的第一个主要块中、第二个子块的第一层偏置为：
 tensor([ 0.0829, -0.3233,  0.4179,  0.1753, -0.1849,  0.4904, -0.3485, -0.2650])


### 2.2 参数初始化

深度学习框架提供默认随机初始化，同样也允许我们创建自定义初始化方法，以满足自定义算法设计的需求。

默认情况下，Pytorch会根据一个范围均匀地初始化权重和偏置矩阵，这个范围是根据输入和输出维度计算出来的。Pytorch的`nn.init`模块提供了多种预置初始化方法。

#### 2.2.1 

# （先总结到这里，后续跟进）